# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [276]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [277]:
portfolio.head(10)

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [278]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [279]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [280]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

#### Transcript file's event columns has four categories: 
* transaction
* offer received
* offer viewed
* offer completed

lets group by according to event type and extract the corresponding values

In [281]:
## Creating separate dataframes for each event type for easy sorting of values
event=transcript.groupby('event')
transaction_df=event.get_group('transaction')
offer_received_df=event.get_group('offer received')
offer_viewed_df=event.get_group('offer viewed')
offer_completed_df=event.get_group('offer completed')

## Values
* Offer Viewed : offer_id
* Offer Received: offer_id
* Offer Completed: offer_id
* transaction: amount

### On using str.replace on 'value' column, it was returning Nan values, therefore google helped me solve this problem [https://stackoverflow.com/questions/50529269/pandas-str-split-returning-nan]

In [282]:
amt=pd.DataFrame(transaction_df['value'].values.tolist(), index=transaction_df.index)
transaction_df=pd.concat([transaction_df, amt], axis=1).drop(columns='value')

In [283]:
offer=pd.DataFrame(offer_viewed_df['value'].values.tolist(), index=offer_viewed_df.index)
offer_viewed_df=pd.concat([offer_viewed_df, offer], axis=1).drop(columns='value')

In [284]:
offer=pd.DataFrame(offer_received_df['value'].values.tolist(), index=offer_received_df.index)
offer_received_df=pd.concat([offer_received_df, offer], axis=1).drop(columns='value')

In [285]:
offer=pd.DataFrame(offer_completed_df['value'].values.tolist(), index=offer_completed_df.index)
offer_completed_df=pd.concat([offer_completed_df, offer], axis=1).drop(columns='value')

In [286]:
transaction_df.rename(columns={'time':'time_completed'}, inplace=True)
transaction_df.head()

,person,event,time_completed,amount
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97


In [287]:
offer_received_df.rename(columns={'offer id':'offer_id'}, inplace=True)
offer_received_df.head()

,person,event,time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [288]:
offer_viewed_df.rename(columns={'offer id':'offer_id'}, inplace=True)
offer_viewed_df.head()

,person,event,time,offer_id
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,0,f19421c1d4aa40978ebb69ca19b0e20d
12651,d1ede868e29245ea91818a903fec04c6,offer viewed,0,5a8bc65990b245e5a138643cd4eb9837
12652,102e9454054946fda62242d2e176fdce,offer viewed,0,4d5c57ea9a6940dd891ad53e9dbe8da0
12653,02c083884c7d45b39cc68e1314fec56c,offer viewed,0,ae264e3637204a6fb9bb56bc8210ddfd
12655,be8a5d1981a2458d90b255ddc7e0d174,offer viewed,0,5a8bc65990b245e5a138643cd4eb9837


In [289]:
offer_completed_df.head()

,person,event,time,offer_id,reward
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,2
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,fafdcd668e3743c1bb461111dcafc2a4,2
12679,629fc02d56414d91bca360decdfa9288,offer completed,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5
12692,676506bad68e4161b9bbaffeb039626b,offer completed,0,ae264e3637204a6fb9bb56bc8210ddfd,10
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10


#### Now we have dataframes of different event types and their corresponding offer_id, reward and amount

In [290]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


### Converting Channel Column in Portfolio into dummy varaibles

### Preparing for Merging of  Offer received, viewed and completed DataFrames to Portfolio DF

In [291]:
portfolio.rename(columns={'id':'offer_id'}, inplace=True)

In [292]:
#converting duration into hours
portfolio['duration']=portfolio['duration']*24

In [293]:
offer_completed_df.head()

,person,event,time,offer_id,reward
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0,2906b810c7d4411798c6938adc9daaa5,2
12672,fe97aa22dd3e48c8b143116a8403dd52,offer completed,0,fafdcd668e3743c1bb461111dcafc2a4,2
12679,629fc02d56414d91bca360decdfa9288,offer completed,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5
12692,676506bad68e4161b9bbaffeb039626b,offer completed,0,ae264e3637204a6fb9bb56bc8210ddfd,10
12697,8f7dd3b2afe14c078eb4f6e6fe4ba97d,offer completed,0,4d5c57ea9a6940dd891ad53e9dbe8da0,10


In [294]:
offer_completed_df=pd.merge(offer_completed_df,portfolio, on='offer_id', how='left').drop(columns='reward_y').rename(columns={'reward_x':'reward','event':'offer_completed'})

In [295]:
offer_received_df=pd.merge(offer_received_df , portfolio, on='offer_id', how='left').rename(columns={'event':'offer_received'}).drop(columns=['reward'])

In [296]:
offer_viewed_df=pd.merge(offer_viewed_df , portfolio, on='offer_id', how='left').rename(columns={'event':'offer_view'}).drop(columns=['reward'])

In [297]:
offer_received_df.head()

,person,offer_received,time,offer_id,channels,difficulty,duration,offer_type
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,"[web, email, mobile]",5,168,bogo
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,"[web, email]",20,240,discount
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,"[web, email, mobile]",10,168,discount
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,"[web, email, mobile, social]",10,240,discount
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,"[web, email, mobile, social]",10,120,bogo


In [298]:
print(offer_received_df.shape, offer_viewed_df.shape, offer_completed_df.shape)

(76277, 8) (57725, 8) (33579, 9)


In [299]:
offer_completed_df['offer_type'].value_counts()

discount    17910
bogo        15669
Name: offer_type, dtype: int64

In [300]:
transaction_df.head()

,person,event,time_completed,amount
12654,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56
12659,54890f68699049c2a04d415abc25e717,transaction,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97


In [301]:
offer_completed_df.sort_values(['person','offer_id'])

,person,offer_completed,time,offer_id,reward,channels,difficulty,duration,offer_type
25942,0009655768c64bdeb2e877511632db8f,offer completed,576,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,168,discount
14758,0009655768c64bdeb2e877511632db8f,offer completed,414,f19421c1d4aa40978ebb69ca19b0e20d,5,"[web, email, mobile, social]",5,120,bogo
22811,0009655768c64bdeb2e877511632db8f,offer completed,528,fafdcd668e3743c1bb461111dcafc2a4,2,"[web, email, mobile, social]",10,240,discount
25955,0011e0d4e6b944f998e987f904e8c1e5,offer completed,576,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,"[web, email]",20,240,discount
8737,0011e0d4e6b944f998e987f904e8c1e5,offer completed,252,2298d6c36e964ae4a3e7e9706d1fb8c2,3,"[web, email, mobile, social]",7,168,discount
...,...,...,...,...,...,...,...,...,...
13021,ffff82501cea40309d5fdd7edcca4a07,offer completed,384,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,168,discount
14693,ffff82501cea40309d5fdd7edcca4a07,offer completed,414,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,168,discount
25850,ffff82501cea40309d5fdd7edcca4a07,offer completed,576,2906b810c7d4411798c6938adc9daaa5,2,"[web, email, mobile]",10,168,discount
20522,ffff82501cea40309d5fdd7edcca4a07,offer completed,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,"[web, email, mobile]",5,168,bogo


In [302]:
offer_received_df.isnull().sum()

person            0
offer_received    0
time              0
offer_id          0
channels          0
difficulty        0
duration          0
offer_type        0
dtype: int64

### Combining all three dataframe (received, viewed and completed ) into one

In [303]:
offer_viewed_df.columns

Index(['person', 'offer_view', 'time', 'offer_id', 'channels', 'difficulty',
       'duration', 'offer_type'],
      dtype='object')

In [304]:
offers=pd.merge(offer_received_df, offer_viewed_df, on=['person','offer_id','offer_type','difficulty','duration'], 
                how='left').rename(columns={'time_x':'time_received','time_y':'time_viewed'}).drop(columns=['channels_y'])
#.drop_duplicates(subset=['person','offer_id','time_x','time_y'])
offers=pd.merge(offers, offer_completed_df, on=['person','offer_id','offer_type','difficulty','duration'], 
                how='left').rename(columns={'channels_x':'channels','time':'time_completed'}).drop(columns=['channels'])
#.drop_duplicates(subset=['person','offer_id','time_x'])
#offers.isnull().sum()

In [305]:
offers.isna().sum()

person                 0
offer_received         0
time_received          0
offer_id               0
difficulty             0
duration               0
offer_type             0
offer_view         16676
time_viewed        16676
offer_completed    48212
time_completed     48212
reward             48212
dtype: int64

In [306]:
offers['validity']=offers['time_received']+offers['duration']

In [307]:
offers=offers.reindex(sorted(offers.columns), axis=1)

### Separating Data frame for offers that went waste
* Offer was received but not viewed and completed
* Offer was received NOT Viewed but Completed (User Purchase was not due to Offer)
* Offer was received, viewed but Not Completed (Excluding offers of offer_type: informational)

In [308]:
offers['offer_type'].value_counts()

bogo             48608
discount         48366
informational    18635
Name: offer_type, dtype: int64

In [309]:
offers['offer_view']=np.where(offers['offer_view'].isna(), 0, 1)
offers['time_viewed']=offers['time_viewed'].fillna(0)

In [310]:
offers['offer_completed']=np.where(offers['offer_view'].isna(), 0, 1)

In [311]:
offers.drop_duplicates(inplace=True)

In [312]:
offers.isna().sum()

difficulty             0
duration               0
offer_completed        0
offer_id               0
offer_received         0
offer_type             0
offer_view             0
person                 0
reward             48212
time_completed     48212
time_received          0
time_viewed            0
validity               0
dtype: int64

In [313]:
#Create a info df where all the offer_type info are stored 
info=offers[offers['offer_type']=='informational']

In [314]:
offers=offers[offers['offer_type']!='informational'].reset_index(drop=True)

### Merging offers DF and transaction DF

In [316]:
offers=pd.merge(offers,transaction_df, on=['person','time_completed'], how='left', copy=False)

In [319]:
offers.drop(columns=['event','duration'], inplace=True)

In [320]:
offers['offer_completed']=np.where((offers['time_received'].le(offers['time_completed']) )&( offers['time_completed'].le(offers['validity'])), 1, 0)

In [324]:
offers

,difficulty,offer_completed,offer_id,offer_received,offer_type,offer_view,person,reward,time_completed,time_received,time_viewed,validity,amount
0,5,1,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer received,bogo,1,78afa995795e4d85b5d9ceeca43f5fef,5.0,132.0,0,6.0,168,19.89
1,20,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,discount,1,a03223e636434f42ac4c3df47e8bac43,NaN,NaN,0,6.0,240,NaN
3,10,0,2906b810c7d4411798c6938adc9daaa5,offer received,discount,1,e2127556f4f64592b11af22de27a7932,NaN,NaN,0,18.0,168,NaN
4,10,0,fafdcd668e3743c1bb461111dcafc2a4,offer received,discount,1,8ec6ce2a7e7949b1bf142def7d0e0586,NaN,NaN,0,12.0,240,NaN
6,10,0,4d5c57ea9a6940dd891ad53e9dbe8da0,offer received,bogo,1,68617ca6246f4fbc85e91a2a49552598,NaN,NaN,0,84.0,120,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95294,20,1,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer received,discount,1,670626b55bfb4ba39c85b27cc7cca527,5.0,666.0,576,624.0,816,14.67
95295,10,1,ae264e3637204a6fb9bb56bc8210ddfd,offer received,bogo,1,d087c473b4d247ccb0abfef59ba12b0e,10.0,636.0,576,672.0,744,20.78
95296,10,0,2906b810c7d4411798c6938adc9daaa5,offer received,discount,0,cb23b66c56f64b109d673d5e56574529,2.0,156.0,576,0.0,744,21.62
95297,7,0,2298d6c36e964ae4a3e7e9706d1fb8c2,offer received,discount,1,6d5f3a774f3d4714ab0c092238f3a1d7,NaN,NaN,576,588.0,744,NaN


In [323]:
offers.drop_duplicates(subset=['person','time_completed','offer_id','time_received'],inplace=True)

In [1570]:
# Dropping all the wasted offers rows

#offer_success=offer_success.drop(offer_success[(offer_success['amount']) < (offer_success['least_pay'])].index)

In [1573]:
waste_offer.drop_duplicates(subset=['person','offer_id','time_completed','time_view','amount'],inplace=True)

### Assumption: Since time_completed associated to Informational type offer is NaN, therefore, assuming that, for all the rows of  Informational type offer, when transaction time is in between time_view and time_validity, it would be considered successful for the particular informational type offer

In [1576]:
# Removing transactions that are assigned to succesful offers DF and Wasted Offers Df
tran1= pd.merge(transaction_df, offer_success, on=['person','time_completed'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True).dropna(axis=1)
tran1.rename(columns={'amount_x':'amount'},inplace=True)

In [1577]:
# Removing transaction that were assigned to waste offers df, since they were done without being Viewed
tran2=pd.merge(tran1, waste_offer_transaction, 
               on=['person','time_completed'], how='outer', 
               indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
tran2= tran2.dropna(subset=['amount_x'])

In [1578]:
tran2=tran2.dropna(axis=1)
tran2.rename(columns={'amount_x':'amount'},inplace=True)

#### Difference between two DF's : Found on :https://stackoverflow.com/questions/44546086/remove-one-dataframe-from-another-with-pandas?rq=1

In [1579]:
info=pd.merge(info, tran2, on=['person'], how='right', copy=False).drop(columns=['event','amount_x'])

In [1580]:
info=info[info['time_completed_y'].ge(info['time_view']) & info['time_completed_y'].le(info['time_validity'])]

In [1581]:
info.drop(columns=['time_completed_x'], inplace=True)
info.rename(columns={'time_completed_y':'time_completed','amount_y':'amount'}, inplace=True)

In [1582]:
#Since Many Informational Offers can be sent to one user and transaction could have been done within the time frame,
#so it would be unfair to assign one amount to success of all informational offers, instead amount will be assigned 
#to just one, but others are only marked as Success but not amount
info.loc[info.duplicated(subset=['offer_id','person','time_view','time_completed','amount','reward']),['amount']]=0

In [1583]:
offer_success=pd.concat([offer_success,info], ignore_index=True).drop_duplicates()

In [1584]:
offer_success.drop_duplicates(subset=['offer_id','person','time_view','time_start'],inplace=True)
offer_success.drop_duplicates(subset=['offer_id','person','time_completed','amount'],inplace=True)

In [1585]:
offer_success.shape

(28587, 20)

In [1586]:
offer_success.sort_values(by=['person','offer_id','amount'])

,channel_email,channel_mobile,channel_social,channel_web,difficulty,duration,least_pay,offer_completed,offer_id,offer_received,offer_type,offer_view,person,reward,time_completed,time_start,time_view,total,time_validity,amount
28484,1.0,1.0,1.0,0.0,0.0,72.0,0.0,0.0,5a8bc65990b245e5a138643cd4eb9837,1.0,informational,2.0,0009655768c64bdeb2e877511632db8f,0.0,228.0,168.0,192.0,3.0,240.0,22.16
12795,1.0,1.0,1.0,1.0,5.0,120.0,5.0,4.0,f19421c1d4aa40978ebb69ca19b0e20d,1.0,bogo,2.0,0009655768c64bdeb2e877511632db8f,5.0,414.0,408.0,456.0,7.0,528.0,8.57
16562,1.0,1.0,1.0,1.0,10.0,240.0,8.0,4.0,fafdcd668e3743c1bb461111dcafc2a4,1.0,discount,2.0,0009655768c64bdeb2e877511632db8f,2.0,528.0,504.0,540.0,7.0,744.0,14.11
12891,1.0,0.0,0.0,1.0,20.0,240.0,15.0,4.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,discount,2.0,0011e0d4e6b944f998e987f904e8c1e5,5.0,576.0,408.0,432.0,7.0,648.0,22.05
5115,1.0,1.0,1.0,1.0,7.0,168.0,4.0,4.0,2298d6c36e964ae4a3e7e9706d1fb8c2,1.0,discount,2.0,0011e0d4e6b944f998e987f904e8c1e5,3.0,252.0,168.0,186.0,7.0,336.0,11.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1.0,1.0,1.0,1.0,5.0,120.0,5.0,4.0,f19421c1d4aa40978ebb69ca19b0e20d,1.0,bogo,2.0,fffad4f4828548d1b5583907f2e9906b,5.0,36.0,0.0,6.0,7.0,120.0,6.97
4552,1.0,0.0,0.0,1.0,20.0,240.0,15.0,4.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,discount,2.0,ffff82501cea40309d5fdd7edcca4a07,5.0,198.0,168.0,174.0,7.0,408.0,22.88
8672,1.0,1.0,0.0,1.0,10.0,168.0,8.0,4.0,2906b810c7d4411798c6938adc9daaa5,1.0,discount,2.0,ffff82501cea40309d5fdd7edcca4a07,2.0,384.0,336.0,354.0,7.0,504.0,15.57
16160,1.0,1.0,0.0,1.0,5.0,168.0,5.0,4.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0,bogo,2.0,ffff82501cea40309d5fdd7edcca4a07,5.0,504.0,504.0,534.0,7.0,672.0,7.79


In [1587]:
offer_success.isnull().sum()

channel_email      0
channel_mobile     0
channel_social     0
channel_web        0
difficulty         0
duration           0
least_pay          0
offer_completed    0
offer_id           0
offer_received     0
offer_type         0
offer_view         0
person             0
reward             0
time_completed     0
time_start         0
time_view          0
total              0
time_validity      0
amount             0
dtype: int64

In [1588]:
offer_success.head()

,channel_email,channel_mobile,channel_social,channel_web,difficulty,duration,least_pay,offer_completed,offer_id,offer_received,offer_type,offer_view,person,reward,time_completed,time_start,time_view,total,time_validity,amount
0,1.0,1.0,0.0,1.0,5.0,168.0,5.0,4.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0,bogo,2.0,78afa995795e4d85b5d9ceeca43f5fef,5.0,132.0,0.0,6.0,7.0,168.0,19.89
1,1.0,1.0,1.0,1.0,5.0,120.0,5.0,4.0,f19421c1d4aa40978ebb69ca19b0e20d,1.0,bogo,2.0,389bc3fa690240e798340f5a15918d5c,5.0,60.0,0.0,0.0,7.0,120.0,9.54
2,1.0,0.0,0.0,1.0,20.0,240.0,15.0,4.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,discount,2.0,d53717f5400c4e84affdaeda9dd926b3,5.0,30.0,0.0,30.0,7.0,240.0,339.69
3,1.0,1.0,1.0,1.0,10.0,240.0,8.0,4.0,fafdcd668e3743c1bb461111dcafc2a4,1.0,discount,2.0,f806632c011441378d4646567f357a21,2.0,126.0,0.0,18.0,7.0,240.0,28.29
4,1.0,0.0,0.0,1.0,20.0,240.0,15.0,4.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,discount,2.0,d058f73bf8674a26a95227db098147b1,5.0,192.0,0.0,30.0,7.0,240.0,27.23


### Now we have all the offers that were successful and their respective amount transactions

### Creating a User based amount spend dataframe (Profile_amount)

In [1589]:
a=transaction_df.groupby('person')['amount'].sum()

In [1590]:
profile.rename(columns={'id':'person'}, inplace=True)

In [1591]:
profile_amount = pd.merge(profile,a, on='person', how='left')

### Cleaning Profile_amount DF

In [1592]:
profile_amount['age']=profile_amount['age'].replace(118,np.nan)

In [1593]:
profile_amount['gender'].fillna(np.nan, inplace=True)

In [1594]:
profile_amount['amount'].fillna(np.nan, inplace=True)

In [1595]:
#Converting joining date into date time format
profile_amount['became_member_on']=pd.to_datetime(profile_amount['became_member_on'], format='%Y%m%d')

In [1596]:
profile_amount.head()

,gender,age,person,became_member_on,income,amount
0,NaN,NaN,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,20.40
1,F,55.0,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,77.01
2,NaN,NaN,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,14.30
3,F,75.0,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,159.27
4,NaN,NaN,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,4.65


### Cleaning of profile_amt df done!

### Transactions that were made without giving offers

In [1597]:
# Finding transactions that were not made under Influence of any offer
transaction_with_offers=offer_success.append(waste_offer).drop_duplicates().reset_index()
normal_transactions=pd.merge(transaction_df, transaction_with_offers, on=['person','time_completed'], how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)

In [1598]:
normal_transactions=normal_transactions.dropna(subset=['time_completed','amount_x']).drop_duplicates()
normal_transactions=normal_transactions.dropna(axis=1)
normal_transactions.rename(columns={'amount_x':'amount'}, inplace=True)

In [1599]:
normal_transactions.head()

,person,event,time_completed,amount
0,02c083884c7d45b39cc68e1314fec56c,transaction,0.0,0.83
1,54890f68699049c2a04d415abc25e717,transaction,0.0,13.23
2,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0.0,19.51
3,bbeb54e861614fc7b22a8844f72dca6c,transaction,0.0,0.22
4,a97e6f33219c432db82acfa0d19c602d,transaction,0.0,18.59


### Exporting DataFrames into CSV Format

In [1600]:
offer_success['offer_completed']=1

In [1601]:
waste_offer.loc[waste_offer['offer_completed']==4, 'offer_completed']=1

In [1602]:
waste_offer_transaction.loc[:,'offer_completed']=1

/Users/aadhar/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [1603]:
waste_offer_transaction

,channel_email,channel_mobile,channel_social,channel_web,difficulty,duration,least_pay,offer_completed,offer_id,offer_received,offer_type,offer_view,person,reward,time_completed,time_start,time_view,total,time_validity,amount
20,1,1,0,1,10,168,8,1,2906b810c7d4411798c6938adc9daaa5,1,discount,0.0,868317b9be554cb18e50bc68484749a2,2,132.0,0,NaN,5.0,168.0,12.03
23,1,1,0,1,10,168,8,1,2906b810c7d4411798c6938adc9daaa5,1,discount,0.0,4beeb3ed64dd4898b0edf2f6b67426d3,2,120.0,0,NaN,5.0,168.0,2.20
62,1,1,0,1,10,168,8,1,2906b810c7d4411798c6938adc9daaa5,1,discount,0.0,8b70ca1f5f5a45bdac2ac2f990b71526,2,168.0,0,NaN,5.0,168.0,29.44
81,1,1,0,1,5,168,5,1,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,bogo,0.0,52f0bd64403f4958ab8fe9a6947c13c3,5,582.0,0,NaN,5.0,168.0,9.20
101,1,0,0,1,20,240,15,1,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,discount,0.0,88baa20c29a94178a43a7d68e5f039d4,5,132.0,0,NaN,5.0,240.0,25.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76050,1,1,0,1,10,168,8,1,2906b810c7d4411798c6938adc9daaa5,1,discount,2.0,cc251fdc5cc24936a741924a785ede5e,2,690.0,576,594.0,7.0,744.0,4.66
76117,1,1,1,1,7,168,4,1,2298d6c36e964ae4a3e7e9706d1fb8c2,1,discount,2.0,cd12501f7dfb46e6a8abe143797ae287,3,696.0,576,624.0,7.0,744.0,2.54
76199,1,0,0,1,20,240,15,1,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,discount,2.0,29b2d42d18e24a68960fdd073d95fb48,5,690.0,576,642.0,7.0,816.0,14.22
76209,1,1,1,1,7,168,4,1,2298d6c36e964ae4a3e7e9706d1fb8c2,1,discount,2.0,9215974995e746d79398eb22de1c9742,3,654.0,576,636.0,7.0,744.0,3.04


In [1604]:
offer_success[offer_success.duplicated(subset=['person','amount','time_completed'])]

,channel_email,channel_mobile,channel_social,channel_web,difficulty,duration,least_pay,offer_completed,offer_id,offer_received,offer_type,offer_view,person,reward,time_completed,time_start,time_view,total,time_validity,amount
4086,1.0,1.0,0.0,1.0,10.0,168.0,8.0,1,2906b810c7d4411798c6938adc9daaa5,1.0,discount,2.0,d058f73bf8674a26a95227db098147b1,2.0,192.0,168.0,180.0,7.0,336.0,27.23
4093,1.0,1.0,0.0,1.0,10.0,168.0,8.0,1,2906b810c7d4411798c6938adc9daaa5,1.0,discount,2.0,9c64321646bf40f6ad1bc98864eaa6b8,2.0,216.0,168.0,174.0,7.0,336.0,20.16
4138,1.0,1.0,1.0,1.0,5.0,120.0,5.0,1,f19421c1d4aa40978ebb69ca19b0e20d,1.0,bogo,2.0,8c410d84af08408fb41f953c93ffac27,5.0,168.0,168.0,204.0,7.0,288.0,11.93
4198,1.0,1.0,1.0,0.0,10.0,168.0,10.0,1,ae264e3637204a6fb9bb56bc8210ddfd,1.0,bogo,2.0,ef0dcb0bddc54f3595846a87bd1237ef,10.0,180.0,168.0,600.0,7.0,336.0,32.03
4230,1.0,1.0,1.0,1.0,5.0,120.0,5.0,1,f19421c1d4aa40978ebb69ca19b0e20d,1.0,bogo,2.0,4324b549f593457696d336c2925ee12f,5.0,168.0,168.0,204.0,7.0,288.0,25.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34489,1.0,1.0,1.0,0.0,0.0,72.0,0.0,1,5a8bc65990b245e5a138643cd4eb9837,1.0,informational,2.0,28ddc2fd0f9344008828d2fd1d846e24,0.0,552.0,576.0,432.0,3.0,648.0,0.12
34525,1.0,1.0,1.0,0.0,0.0,72.0,0.0,1,5a8bc65990b245e5a138643cd4eb9837,1.0,informational,2.0,f07862abca4a4b8d98e0ef8ed8b808f5,0.0,504.0,504.0,504.0,3.0,576.0,25.78
36096,1.0,1.0,0.0,1.0,0.0,96.0,0.0,1,3f207df678b143eea3cee63160fa8bed,1.0,informational,2.0,8142c6f808944e7a87a18950c71b18ae,0.0,576.0,576.0,576.0,3.0,672.0,5.80
36568,1.0,1.0,1.0,0.0,0.0,72.0,0.0,1,5a8bc65990b245e5a138643cd4eb9837,1.0,informational,2.0,8a647583b38f4397a8870b70eaefcc08,0.0,594.0,576.0,588.0,3.0,648.0,1.25


In [1605]:
normal_transactions.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/normal_transaction.csv') # trasactions without offer influence
offer_success.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/offer_success.csv', encoding='utf-8') #transactions and other data of successful offers
waste_offer_transaction.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/waste_offer_transaction.csv', encoding='utf-8')  #transactions data of transactions made without viewing offer
waste_offer.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/waste_offer.csv', encoding='utf-8') #total waste offers including waste_offer_transactions
profile_amount.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/profile_amount.csv', encoding='utf-8') #profile and total amount spend data
transaction_df.to_csv(r'/Users/aadhar/Desktop/Udacity_DataScience/Project-4/clean_data/transaction_df.csv', encoding='utf-8') #original Transaction DF